## FineTuning LLM with Model-As-Platform.

This sample shows how to create a standalone FineTuning job to fine tune a model to summarize a dialog between 2 people using samsum dataset. This notebook expects user will provide compute in the form of aml compute cluster or compute sku.

#### Training data
We use sample data placed along with the notebook for our finetuning in files "train.jsonl" and "validation.jsonl"

#### Model
We will use the Phi-3-mini-4k-instruct model to show how user can finetune a model for chat-completion task. If you opened this notebook from a specific model card, remember to replace the specific model name. 

#### Outline
1. Setup pre-requisites
2. Pick a model to fine-tune.
3. Create training and validation datasets.
4. Configure the fine tuning job.
5. Submit the fine tuning job.
6. Create online endpoint deployment using finetuned model and sample inference.

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry
* Set an optional experiment name

**Install dependencies by running below cell. This is not an optional step if running in a new environment.**

In [ ]:
%pip install azure-ai-ml
%pip install azure-identity

%pip install mlflow
%pip install azureml-mlflow

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)
from azure.ai.ml.constants._common import AssetTypes

### Create AzureML Workspace connections

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

try:
    workspace_ml_client = MLClient.from_config(credential=credential)
except:
    workspace_ml_client = MLClient(
        credential,
        subscription_id="<SUBSCRIPTION_ID>",
        resource_group_name="<RESOURCE_GROUP>",
        workspace_name="<WORKSPACE_NAME>",
    )

# the models, fine tuning pipelines and environments are available in various AzureML system registries,
# Example: Phi family of models are in "azureml"
registry_ml_client = MLClient(credential, registry_name="azureml")

# Get AzureML workspace object.
workspace = workspace_ml_client._workspaces.get(workspace_ml_client.workspace_name)
workspace.id

### 2. Pick a model to fine tune

`Phi-3-mini-4k-instruct` is a 3.8B parameters, lightweight, state-of-the-art open model built upon datasets used for Phi-2. The model belongs to the Phi-3 model family, and the Mini version comes in two variants 4K and 128K which is the context length (in tokens) it can support. You can browse these models in the Model Catalog in the Azure AI Studio, filtering by the `chat-completion` task. In this example, we use the `Phi-3-mini-4k-instruct` model. If you have opened this notebook for a different model, replace the model name and version accordingly.

Note the model id property of the model. This will be passed as input to the fine tuning job. This is also available as the `Asset ID` field in model details page in Azure AI Studio Model Catalog.

In [ ]:
model_name = "Phi-3-mini-4k-instruct"
model_to_finetune = registry_ml_client.models.get(model_name, label="latest")
print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for fine tuning".format(
        model_to_finetune.name, model_to_finetune.version, model_to_finetune.id
    )
)

#### Use either compute cluster or provide instance type which is compatible with below list

In [ ]:
model_to_finetune.properties["finetune-recommended-sku"]

### 3. Sample data

The chat-completion dataset is stored in parquet format with each entry using the following schema:


    {
        "messages":[",
            {",
                "content": "Create a fully-developed protagonist who is challenged to survive within a dystopian society under the rule of a tyrant. ...",
                "role": "user",
            },
            {",
                "content": "Name: Ava\n Ava was just 16 years old when the world as she knew it came crashing down. The government had collapsed, leaving behind a chaotic and lawless society. ...",
                "role": "assistant",
            },
            {",
                "content": "Wow, Ava's story is so intense and inspiring! Can you provide me with more details.  ...",
                "role": "user",
            },
            {
                "content": "Certainly! ....",
                "role": "assistant"",
            }
        ],
    }

#### Create data inputs

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Data

dataset_version = "1"
train_dataset_name = "chat_training_small"
try:
    train_data_asset = workspace_ml_client.data.get(
        train_dataset_name, version=dataset_version
    )
    print(f"Dataset {train_dataset_name} already exists")
except:
    print("creating dataset")
    train_data = Data(
        path=f"./train.jsonl",
        type=AssetTypes.URI_FILE,
        description="Training dataset",
        name=train_dataset_name,
        version="1",
    )
    train_data_asset = workspace_ml_client.data.create_or_update(train_data)

In [ ]:
train_data_asset.id

In [ ]:
from azure.ai.ml.entities import Data

dataset_version = "1"
validation_dataset_name = "chat_validation_small"
try:
    validation_data_asset = workspace_ml_client.data.get(
        validation_dataset_name, version=dataset_version
    )
    print(f"Dataset {validation_dataset_name} already exists")
except:
    print("creating dataset")
    validation_data = Data(
        path=f"./validation.jsonl",
        type=AssetTypes.URI_FILE,
        description="Validation dataset",
        name=validation_dataset_name,
        version="1",
    )
    validation_data_asset = workspace_ml_client.data.create_or_update(validation_data)

In [ ]:
validation_data_asset.id

### 3. Submit the fine tuning job using the the model and data as inputs
 
Create FineTuning job using all the data that we have so far.

##### Create marketplace subscription for 3P models
Note: Skip this step for 1P(Microsoft) models that are offered on Azure. Example: Phi family of models

#### Define finetune parameters

##### There are following set of parameters that are required.

1. `name`- FineTuning job name
2. `model` - Base model to finetune.
3. `training_data` - Training data for finetuning the base model.
4. `task` - FineTuning task to perform. eg. TEXT_COMPLETION for text-generation/text-generation finetuning jobs.
5. `outputs`- Output registered model name.

##### Following parameters are optional:

1. `hyperparameters` - Parameters that control the FineTuning behavior at runtime.
2. `validation_data` - Validation data for finetuning the base model.
3. `experiment_name` - Experiment name for FineTuning job.
4. `display_name` - FineTuning job display name.

In [ ]:
from azure.ai.ml.finetuning import FineTuningTaskType, create_finetuning_job
import uuid

guid = uuid.uuid4()
short_guid = str(guid)[:8]
display_name = f"{model_name}-display-name-{short_guid}"
name = f"finetuned-model-{short_guid}"
output_model_name_prefix = f"finetuned-model-{short_guid}"
experiment_name = f"finetuning-llm"
compute = "gpu-compute-low-pri"

finetuning_job = create_finetuning_job(
    task=FineTuningTaskType.CHAT_COMPLETION,
    training_data=train_data_asset.id,
    validation_data=validation_data_asset.id,
    hyperparameters={
        "per_device_train_batch_size": "1",
        "learning_rate": "0.00002",
        "num_train_epochs": "1",
    },
    model=model_to_finetune.id,
    display_name=display_name,
    name=name,
    experiment_name=experiment_name,
    # compute=compute,
    instance_types=["Standard_ND96amsr_A100_v4", "Standard_E4s_v3"],
    tags={"foo_tag": "bar"},
    properties={"my_property": "my_value"},
    output_model_name_prefix=output_model_name_prefix,
)

In [ ]:
created_job = workspace_ml_client.jobs.create_or_update(finetuning_job)
workspace_ml_client.jobs.get(created_job.name)

#### Wait for the above job to complete successfully

In [ ]:
status = workspace_ml_client.jobs.get(created_job.name).status

import time

while True:
    status = workspace_ml_client.jobs.get(created_job.name).status
    print(f"Current job status: {status}")
    if status in ["Failed", "Completed", "Canceled"]:
        print("Job has finished with status: {0}".format(status))
        break
    else:
        print("Job is still running. Checking again in 30 seconds.")
        time.sleep(30)

In [ ]:
if status in ["Failed", "Canceled"]:
    print(
        "Job did not finish successfully. So no model to deploy. JobStatus: {0}".format(
            status
        )
    )
    break

In [ ]:
registered_model_output = created_job.outputs["registered_model"]
registered_model_output

In [ ]:
registered_model = workspace_ml_client.models.get(
    name=registered_model_output.name, version="1"
)

In [ ]:
registered_model

### 4. Deploy the fine tuned model to an online endpoint

In [ ]:
import time, sys, re
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    ProbeSettings,
    OnlineRequestSettings,
)

sanitized_model_name_for_deployment = re.sub(r"[^a-zA-Z0-9-]", "-", model_name)
online_endpoint_name = f"{sanitized_model_name_for_deployment}-" + short_guid
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for "
    + registered_model_output.name
    + ", fine tuned model for samsum textgen",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

#### Create a deployment

#### You can find the SKUS supported by the model for deployment with below command. You can choose one of the sku for deployment.

In [ ]:
foundation_model.properties["inference-recommended-sku"]

In [ ]:
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_NC24ads_A100_v4",
    instance_count=1,
    liveness_probe=ProbeSettings(initial_delay=600),
    request_settings=OnlineRequestSettings(request_timeout_ms=90000),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

In [ ]:
import requests

url = f"{endpoint.scoring_uri}/v1/chat/completions"

payload = {
    "max_tokens": 1024,
    "messages": [
        {
            "content": "This script is great so far. Can you add more dialogue between Amanda and Thierry to build up their chemistry and connection?",
            "role": "user",
        }
    ],
}
headers = {"Content-Type": "application/json", "Authorization": f"{auth_key}"}

response = requests.post(url, json=payload, headers=headers)

response.json()